In [9]:
import folium
import pandas as pd
import numpy as np
from sklearn import linear_model

Read Data
==

In [10]:
df = pd.read_csv('resources/data/LocTreino_Equipe_2.csv')
df = df.dropna(axis='index')
lats = df['lat']
lons = df['lon']
positions = zip(lats, lons)

In [11]:
df_bts = pd.read_csv('resources/data/dados_BTSs.csv')
bts_lats = df_bts['lat']
bts_lons = df_bts['lon']
bts_positions = zip(bts_lats, bts_lons)

In [12]:
for column in df.columns[3:9]:
    print(column)

pathBTS1
pathBTS2
pathBTS3
pathBTS4
pathBTS5
pathBTS6


## Plot Original Points


In [13]:
center_lat = np.mean([lat for lat in list(df['lat'])])
center_lon = np.mean([lon for lon in list(df['lon'])])
original_pts_map = folium.Map(location=[center_lat, center_lon],
                              zoom_start=15)

In [14]:
for location in positions:
    marker = folium.CircleMarker(location=location,
                                 radius=1,
                                 color='blue',
                                 fill_opacity=0.1)
    marker.add_to(original_pts_map)
for location in bts_positions:
    marker = folium.Marker(location=location)
    marker.add_to(original_pts_map)
original_pts_map.save("resources/maps/original_pts_map.html")

In [15]:
original_pts_map

## Fit Linear Model

In [16]:
samples = df[df.columns[3:]].values
target_lat = df['lat'].values
target_lon = df['lon'].values

In [17]:
lat_reg = linear_model.LinearRegression(normalize=True)
lat_reg.fit(samples, target_lat)

lon_reg = linear_model.LinearRegression(normalize=True)
lon_reg.fit(samples, target_lon)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=True)

### Coefficients

In [18]:
sorted(zip(df.columns[3:], lat_reg.coef_),
       key=lambda (_, x): abs(x),
       reverse=True)

SyntaxError: invalid syntax (<ipython-input-18-86d7fd356772>, line 2)

In [ ]:
predicted_lat = lat_reg.predict(samples)
predicted_lon = lon_reg.predict(samples)

### Calculate Error

In [ ]:
# for p_lat, p_long, (lat, long) in list(zip(predicted_lat, predicted_long, targets)):
#     print(100*(p_lat - lat)/lat, 100*(p_long - long)/long)
# plt.plot(predicted_lat, [x for x, _ in targets], marker='x', linestyle='None')
# plt.show()
from sklearn.metrics import mean_squared_error as mse
from math import sqrt

def rmse(y_pred, y_real):
    return sqrt(mse(y_pred, y_real))

print('Latitude Error:', rmse(predicted_lat, target_lat))
print('Longitude Error:', rmse(predicted_long, target_lon))

In [ ]:
result_map = folium.Map(location=[center_lat, center_lon],
                        zoom_start=13,
                        tiles='CartoDB dark_matter')
n_points = 10000
predicted_positions = list(zip(predicted_lat, predicted_long))
for location in predicted_positions[:n_points]:
    marker = folium.CircleMarker(location=location, color='red', weight=1, radius=3)
    marker.add_to(result_map)
for location in positions[:n_points]:
    marker = folium.CircleMarker(location=location, color='blue', weight=1, radius=3)
    marker.add_to(result_map)
for location in bts_positions:
    marker = folium.Marker(location=location)
    marker.add_to(result_map)
for p1, p2 in list(zip(positions, predicted_positions))[:n_points]:
    folium.PolyLine([p1, p2], color="green", weight=1, opacity=0.5).add_to(result_map)
result_map.save("resources/maps/result_map.html")

## Fit KNN Model

In [19]:
from sklearn import neighbors

In [20]:
lat_reg = neighbors.KNeighborsRegressor(n_neighbors=1)
long_reg = neighbors.KNeighborsRegressor(n_neighbors=1)

In [21]:
lat_reg.fit(samples, target_lat)
long_reg.fit(samples, target_lon)

KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
          metric_params=None, n_jobs=1, n_neighbors=1, p=2,
          weights='uniform')

In [22]:
predicted_lat = lat_reg.predict(samples)
predicted_long = long_reg.predict(samples)

In [23]:
# from folium.plugins import MarkerCluster
folium_map = folium.Map(location=[center_lat, center_lon],
                        zoom_start=13,
                        tiles='CartoDB dark_matter')
# cluster = MarkerCluster(list(zip(predicted_lat, predicted_long))).add_to(folium_map)
n_points = -1
predicted_positions = list(zip(predicted_lat, predicted_long))
for location in predicted_positions[:n_points]:
    marker = folium.CircleMarker(location=location,
                                 color='red',
                                 weight=1,
                                 radius=3)
    marker.add_to(folium_map)
for location in positions[:n_points]:
    marker = folium.CircleMarker(location=location,
                                 color='blue',
                                 weight=1,
                                 radius=3)
    marker.add_to(folium_map)
for location in bts_positions:
    marker = folium.Marker(location=location)
    marker.add_to(folium_map)
for p1, p2 in list(zip(positions, predicted_positions))[:n_points]:
    folium.PolyLine([p1, p2],
                    color="green",
                    weight=1,
                    opacity=0.5).add_to(folium_map)
folium_map.save("resources/maps/knn_result_map.html")

TypeError: 'zip' object is not subscriptable

In [24]:
# for p_lat, p_long, (lat, long) in list(zip(predicted_lat, predicted_long, targets)):
#     print(100*(p_lat - lat)/lat, 100*(p_long - long)/long)
# plt.plot(predicted_lat, [x for x, _ in targets], marker='x', linestyle='None')
# plt.show()
from sklearn.metrics import mean_squared_error as mse
from math import sqrt

def rmse(y_pred, y_real):
    return sqrt(mse(y_pred, y_real))

print(rmse(predicted_lat, target_lat))
print(rmse(predicted_long, target_lon))

0.0
0.0


In [25]:
from sklearn.model_selection import cross_validate

In [26]:
cv_results = cross_validate(lat_reg,
                            samples,
                            target_lat,
                            cv=10,
                            return_train_score=False)

In [27]:
cv_results['test_score']

array([0.95623138, 0.94955856, 0.95109055, 0.96697754, 0.95777126,
       0.93641567, 0.95880014, 0.93203664, 0.96924516, 0.96055802])

Fingerprint
==

In [28]:
for column in list(enumerate(df.columns)):
    print(column)
X = df[df.columns[1:3]].values
Y = df[df.columns[3]].values

(0, 'pontoId')
(1, 'lat')
(2, 'lon')
(3, 'pathBTS1')
(4, 'pathBTS2')
(5, 'pathBTS3')
(6, 'pathBTS4')
(7, 'pathBTS5')
(8, 'pathBTS6')
(9, 'taBTS1')
(10, 'taBTS2')
(11, 'taBTS3')
(12, 'taBTS4')
(13, 'taBTS5')
(14, 'taBTS6')


In [29]:
rssi_reg = linear_model.LinearRegression(normalize=True)
rssi_reg.fit(X, Y)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=True)

In [30]:
point = df[df.columns[1:3]].values[0]
rssi = rssi_reg.predict([point])
print(rssi, df[df.columns[3]].values[0])

[110.93807735] 122.556666666667


Creating Regular Squared Grid
==

In [31]:
latitudes = np.array(list(df['lat']))
longitudes = np.array(list(df['lon']))
max_latitude = latitudes.max()
min_latitude = latitudes.min()
max_longitude = longitudes.max()
min_longitude = longitudes.min()

In [32]:
marker = folium.Marker(location=(max_latitude, max_longitude))
marker.add_to(folium_map)
marker = folium.Marker(location=(max_latitude, min_longitude))
marker.add_to(folium_map)
marker = folium.Marker(location=(min_latitude, max_longitude))
marker.add_to(folium_map)
marker = folium.Marker(location=(min_latitude, min_longitude))
marker.add_to(folium_map)

In [33]:
folium_map

In [34]:
number_of_squares = 10
grid_latitudes = np.linspace(min_latitude, max_latitude,number_of_squares)
grid_longitudes = np.linspace(min_longitude, max_longitude,number_of_squares)

In [35]:
for lat in grid_latitudes:
    for lon in grid_longitudes:
        marker = folium.CircleMarker(location=(lat,lon),color='blue',weight=1,radius=3)
        marker.add_to(folium_map)
lat_centers = (grid_latitudes[:-1] + grid_latitudes[1:]) / 2
lon_centers = (grid_longitudes[:-1] + grid_longitudes[1:]) / 2
for lat in lat_centers:
    for lon in lon_centers:
        marker = folium.CircleMarker(location=(lat,lon),color='green',weight=1,radius=3)
        marker.add_to(folium_map)
folium_map

In [36]:
target_positions = []
for lat in lat_centers:
    for lon in lon_centers:
        target_positions.append((lat,lon))
target_positions

[(-8.07655711111111, -34.90562094444444),
 (-8.07655711111111, -34.90320283333333),
 (-8.07655711111111, -34.90078472222222),
 (-8.07655711111111, -34.89836661111111),
 (-8.07655711111111, -34.8959485),
 (-8.07655711111111, -34.89353038888889),
 (-8.07655711111111, -34.89111227777778),
 (-8.07655711111111, -34.88869416666667),
 (-8.07655711111111, -34.886276055555555),
 (-8.074531333333333, -34.90562094444444),
 (-8.074531333333333, -34.90320283333333),
 (-8.074531333333333, -34.90078472222222),
 (-8.074531333333333, -34.89836661111111),
 (-8.074531333333333, -34.8959485),
 (-8.074531333333333, -34.89353038888889),
 (-8.074531333333333, -34.89111227777778),
 (-8.074531333333333, -34.88869416666667),
 (-8.074531333333333, -34.886276055555555),
 (-8.072505555555555, -34.90562094444444),
 (-8.072505555555555, -34.90320283333333),
 (-8.072505555555555, -34.90078472222222),
 (-8.072505555555555, -34.89836661111111),
 (-8.072505555555555, -34.8959485),
 (-8.072505555555555, -34.8935303888888

In [37]:
predicted_rssis1 = rssi_reg.predict(target_positions)
predicted_rssis1

array([113.02456863, 112.21904583, 111.41352303, 110.60800023,
       109.80247743, 108.99695463, 108.19143183, 107.38590903,
       106.58038622, 115.49542396, 114.68990116, 113.88437836,
       113.07885556, 112.27333276, 111.46780996, 110.66228715,
       109.85676435, 109.05124155, 117.96627929, 117.16075649,
       116.35523369, 115.54971089, 114.74418809, 113.93866528,
       113.13314248, 112.32761968, 111.52209688, 120.43713462,
       119.63161182, 118.82608902, 118.02056621, 117.21504341,
       116.40952061, 115.60399781, 114.79847501, 113.99295221,
       122.90798995, 122.10246715, 121.29694434, 120.49142154,
       119.68589874, 118.88037594, 118.07485314, 117.26933034,
       116.46380754, 125.37884527, 124.57332247, 123.76779967,
       122.96227687, 122.15675407, 121.35123127, 120.54570847,
       119.74018567, 118.93466287, 127.8497006 , 127.0441778 ,
       126.238655  , 125.4331322 , 124.6276094 , 123.8220866 ,
       123.0165638 , 122.211041  , 121.4055182 , 130.32

In [40]:
regular_squared_grid = list(zip(target_positions,predicted_rssis1))
regular_squared_grid

[((-8.07655711111111, -34.90562094444444), 113.02456862856775),
 ((-8.07655711111111, -34.90320283333333), 112.21904582808452),
 ((-8.07655711111111, -34.90078472222222), 111.4135230276031),
 ((-8.07655711111111, -34.89836661111111), 110.60800022712169),
 ((-8.07655711111111, -34.8959485), 109.8024774266421),
 ((-8.07655711111111, -34.89353038888889), 108.99695462616069),
 ((-8.07655711111111, -34.89111227777778), 108.19143182567927),
 ((-8.07655711111111, -34.88869416666667), 107.38590902519786),
 ((-8.07655711111111, -34.886276055555555), 106.58038622471645),
 ((-8.074531333333333, -34.90562094444444), 115.49542395780622),
 ((-8.074531333333333, -34.90320283333333), 114.68990115732299),
 ((-8.074531333333333, -34.90078472222222), 113.88437835684158),
 ((-8.074531333333333, -34.89836661111111), 113.07885555636017),
 ((-8.074531333333333, -34.8959485), 112.27333275588057),
 ((-8.074531333333333, -34.89353038888889), 111.46780995539916),
 ((-8.074531333333333, -34.89111227777778), 110.6